In [ ]:
%cd ../

# Utils
import os, sys
import src.data
import src.model

## Set **seed**

In [ ]:
src.data.set_all_seeds()

## Data

### **Initial**

In [5]:
classes = []

In [7]:
image_paths = []
labels = []

### **Split**

In [11]:
from sklearn.model_selection import train_test_split

train_image_paths, valid_image_paths, train_labels, valid_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42) # stratify=labels

### **Augmentations**

In [ ]:
import torchvision.transforms as T

src.data.ImageClassificationDataset.augmentation = T.Compose([
    # Добавьте своих аугментаций (по желанию)
])

### Create **Datasets**

In [ ]:
dataset = src.data.ImageClassificationDataset(image_paths, labels)

train_set = src.data.ImageClassificationDataset(train_image_paths, train_labels, True)
valid_set = src.data.ImageClassificationDataset(valid_image_paths, valid_labels)

### Create **DataLoader**

In [13]:
from torch.utils.data import DataLoader

batch_size = 32
num_workers = 0 # ! Возникают ошибки

train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, num_workers=num_workers, shuffle=False)

### ***Visualization***

In [ ]:
src.data.show_image_classification(dataset, classes=classes)

## **Models**

In [15]:
from torch import nn, optim
from torchvision import models
from transformers import AutoModelForImageClassification

In [18]:
class CustomOutput(nn.Module):
    def __init__(self, model, output_transform=lambda out: out.logits):
        super().__init__()
        self.model = model
        self.output_transform = output_transform

    def forward(self, *args, **kwargs):
        return self.output_transform(self.model(*args, **kwargs))

    def __getattr__(self, name):
        if name in ('model', 'output_transform'):
            return super().__getattr__(name)
        return getattr(self.model, name)
    
    def __setattr__(self, name, value):
        if name in ('model', 'output_transform'):
            super().__setattr__(name, value)
        else:
            setattr(self.model, name, value)

### *Score*

In [19]:
scores = dict()

### **Model**: `EfficientNet_B0`

In [ ]:
model_image_size = (224, 224)
src.data.ImageClassificationDataset.change_image_size(model_image_size)

In [ ]:
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, len(classes))

model_wrapped = src.model.Classifier(model, "EfficientNet_B0")

In [ ]:
model_wrapped.fit(train_loader, valid_loader, 3)

In [ ]:
scores[model_wrapped.best_score] = model_wrapped

### **Model**: `google/vit-base-patch16-224`

In [ ]:
model_image_size = (224, 224)
src.data.ImageClassificationDataset.change_image_size(model_image_size)

In [ ]:
model = CustomOutput(AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224", num_labels=len(classes), ignore_mismatched_sizes=True))
optimizer = optim.Adam(model.parameters(), lr=5e-5)

model_wrapped = src.model.Classifier(model, "Google-VitBase", optimizer)

In [ ]:
model_wrapped.fit(train_loader, valid_loader, 3)

In [ ]:
scores[model_wrapped.best_score] = model_wrapped

## Result

In [ ]:
best_model_wrapped = scores[max(scores)]
best_model_wrapped.name

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

n = (2, 4)  # Кортеж (rows, cols)
fig_image_size = 5

fig, axes = plt.subplots(n[0], n[1], figsize=(fig_image_size * n[1], fig_image_size * n[0]))

for i in range(n[0]):
    for j in range(n[1]):
        idx = random.randrange(len(valid_set))
        batch = valid_set[idx]
        image = valid_set.get_item(idx)['image'].resize((512, 512))
        prediction = best_model_wrapped.predict(batch)

        ax = axes[i][j]
        ax.imshow(np.array(image))
        ax.axis('off')
        ax.set_title(f"Class: {classes[batch['labels']]}\nPredict: {classes[prediction]}", fontsize=10)

plt.tight_layout()
plt.show()

## Submission

In [ ]:
test_dir = ""

test_image_names = os.listdir(test_dir)
test_image_paths = [f"{test_dir}/{image_name}" for image_name in test_image_names]
test_set = src.data.ImageClassificationDataset(test_image_paths)

In [ ]:
predict_class_id = best_model_wrapped.predict(test_set)
predict_class_names = [classes[class_id] for class_id in predict_class_id]